# DLT Notebook - Gold Layer

In [0]:
from pyspark.sql.functions import *

In [0]:
looktables_rules = {
    "rule1" : "show_id IS NOT NULL"
}

In [0]:
@dlt.table(
    name = "gold_netflixdirectors"
)

@dlt.expect_all_or_drop(looktables_rules)
def gold_netflixdirectors():
    df = spark.readStream.format("delta").load("abfss://silver@storagenetflixde.dfs.core.windows.net/netflix_directors")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcast"
)

@dlt.expect_all_or_drop(looktables_rules)
def gold_netflixcast():
    df = spark.readStream.format("delta").load("abfss://silver@storagenetflixde.dfs.core.windows.net/netflix_cast")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcountries"
)

@dlt.expect_all_or_drop(looktables_rules)
def gold_netflixcountries():
    df = spark.readStream.format("delta").load("abfss://silver@storagenetflixde.dfs.core.windows.net/netflix_countries")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcategory"
)

@dlt.expect_or_drop("rule1" ,"show_id IS NOT NULL")
def gold_netflixcategory():
    df = spark.readStream.format("delta").load("abfss://silver@storagenetflixde.dfs.core.windows.net/netflix_category")
    return df

In [0]:
@dlt.table

def gold_stg_netflixtitles():
    df = spark.readStream.format("delta").load("abfss://silver@storagenetflixde.dfs.core.windows.net/netflix_titles")
    return df

In [0]:
@dlt.view

def gold_trans_netflixtitles():
    df = spark.readStream.table("LIVE.gold_stg_netflixtitles")
    df = df.withColumn("newflag", lit(1))
    return df

In [0]:
master_data_rule = {
    "rule1" : "newflag IS NOT NULL",
    "rule2" : "show_id IS NOT NULL"
}

In [0]:
@dlt.table

@dlt.expect_all_or_drop(master_data_rule)
def gold_netflixtitles():
    df = spark.readStream.table("LIVE.gold_trans_netflixtitles")
    return df